In [0]:
%pip install -U --quiet databricks-sdk==0.40.0 databricks-agents==0.16.0 databricks-langchain==0.3.0 mlflow[databricks]==2.20.2 databricks-vectorsearch==0.49 langchain==0.3.19 langchain_core==0.3.37 bs4==0.0.2 markdownify==0.14.1 grpcio-status==1.59.3 # Temporary pin: grpcio version to avoid protobuf conflict.
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%sql
-- The dataset for your knowledge base you wan.
SELECT * FROM dhbb.bronze.dhbb_bronze limit 50;

id file_name content 1 1.text ---
title: COELHO, Machado
natureza: biográfico
sexo: m
cargos:
 - dep. fed. DF 1927-1929 
 - dep. fed. DF 1930
 - const. 1946
 - dep. fed. SP 1946-1951
---

«José Machado Coelho de Castro» nasceu em Lorena (SP).

Estudou no Ginásio Diocesano de São Paulo e bacharelou-se em 1910 pela
Faculdade de Ciências Jurídicas e Sociais. Dedicando-se à advocacia, foi
promotor público em Cunha (SP) e depois delegado de polícia no Rio de
Janeiro, então Distrito Federal.

Iniciou sua vida política como deputado federal pelo Distrito Federal,
exercendo o mandato de 1927 a 1929. Reeleito para a legislatura iniciada
em maio de 1930, ocupava sua cadeira na Câmara quando, em 3 de outubro,
foi deflagrado o movimento revolucionário liderado por Getúlio Vargas.
Ligado ao governo federal, encontrava-se ao lado do presidente
Washington Luís, no palácio Guanabara, no momento de sua deposição no
dia 24 de outubro. Junto com outros companheiros também solidários ao
regime deposto e que se haviam asilado em embaixadas e legações, foi
enviado em novembro para o estrangeiro. Em outubro de 1932, estava
presente no porto de Alcântara, em Lisboa, para receber os
revolucionários constitucionalistas exilados pelo governo de Getúlio
Vargas após a derrota da revolução irrompida em julho desse ano em São
Paulo.

Com a redemocratização do país em 1945, candidatou-se pelo estado de São
Paulo, na legenda do Partido Social Democrático (PSD), às eleições para
a Assembleia Nacional Constituinte (ANC) realizadas em dezembro desse
ano. Obteve uma suplência e, em julho de 1946, foi convocado para
participar dos trabalhos constituintes. Com a promulgação da nova Carta
(18/9/1946) e a transformação da Constituinte em Congresso ordinário,
integrou a Comissão Permanente de Obras Públicas da Câmara Federal,
tendo votado em janeiro de 1948 a favor da cassação dos mandatos dos
parlamentares comunistas. Deixou a Câmara em janeiro de 1951.

Foi ainda presidente da Companhia de Cimento Vale do Paraíba.

Faleceu no Rio de Janeiro no dia 17 de maio de 1975.

 10 10.text ---
title: ABÍLIO, Armando 
natureza: biográfico
sexo: m
cargos: 
 - dep. fed. PB 1995-1999
 - dep. fed. PB 1999-2003
 - dep. fed. PB 2007-2011
 - dep. fed. PB 2012 
autor: 
 - Alan Carneiro 
 - Demian Melo (atualização)
---

«Armando Abílio Vieira» nasceu em Itaporanga (PB) no dia 29 de
dezembro de 1944, filho de Argemiro Abílio de Sousa e de Luísa
Bronzeado Vieira.

Em 1962 ingressou, em Recife, na Faculdade de Ciências Médicas da
Universidade Federal de Pernambuco, pela qual se graduou em
1968. Durante o ano seguinte, frequentou a residência médica no
Hospital Osvaldo Cruz, também na capital pernambucana.

Filiando-se, em 1982, ao Partido Democrático Social (PDS), foi
escolhido para integrar a chapa do partido como candidato a
vice-prefeito de Esperança (PB). Ainda em 1982 fora nomeado diretor do
Hospital Geral da cidade, função que desempenharia até
1991. Elegendo-se no pleito de novembro de 1982, foi empossado no
cargo em janeiro de 1983. Desligando-se do PDS em 1988, permaneceu
como vice-prefeito até janeiro do ano seguinte. Em 1989, filiou-se ao
Partido do Movimento Democrático Brasileiro (PMDB), no qual permaneceu
até 1990.

Ingressando, em seguida, no Partido da Frente Liberal (PFL), em
outubro deste ano elegeu-se deputado estadual nessa legenda. Assumindo
sua cadeira na Assembleia Legislativa paraibana em janeiro de 1991,
foi escolhido segundo vice-presidente da mesa da casa, ocupando essa
função até 1992. Em 1993, retornou ao PMDB, tornando-se ainda neste
ano primeiro vice-presidente da mesa, cargo que desempenharia até o
final da legislatura.

No pleito de outubro de 1994, elegeu-se deputado federal pelo PMDB,
tendo como base eleitoral as regiões do Brejo Paraibano e do vale do
Piancó, no sertão. Concluindo seu mandato na Assembleia Legislativa
paraibana em janeiro de 1995, assumiu sua cadeira na Câmara dos
Deputados em fevereiro seguinte. Tornou-se membro titular da Comissão
de


## Vector search Endpoints

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-prep-2.png?raw=true" style="float: right; margin-left: 10px" width="400px">

Vector search endpoints are entities where your indexes will live. Think about them as entry point to handle your search request. 

Let's start by creating our first Vector Search endpoint. Once created, you can view it in the [Vector Search Endpoints UI](#/setting/clusters/vector-search). Click on the endpoint name to see all indexes that are served by the endpoint.

In [0]:
import time

def endpoint_exists(vsc, vs_endpoint_name):
  try:
    return vs_endpoint_name in [e['name'] for e in vsc.list_endpoints().get('endpoints', [])]
  except Exception as e:
    #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
    if "REQUEST_LIMIT_EXCEEDED" in str(e):
      print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. The demo will consider it exists")
      return True
    else:
      raise e

def wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name):
  for i in range(180):
    try:
      endpoint = vsc.get_endpoint(vs_endpoint_name)
    except Exception as e:
      #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
      if "REQUEST_LIMIT_EXCEEDED" in str(e):
        print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. Please manually check your endpoint status")
        return
      else:
        raise e
    status = endpoint.get("endpoint_status", endpoint.get("status"))["state"].upper()
    if "ONLINE" in status:
      return endpoint
    elif "PROVISIONING" in status or i <6:
      if i % 20 == 0: 
        print(f"Waiting for endpoint to be ready, this can take a few min... {endpoint}")
      time.sleep(10)
    else:
      raise Exception(f'''Error with the endpoint {vs_endpoint_name}. - this shouldn't happen: {endpoint}.\n Please delete it and re-run the previous cell: vsc.delete_endpoint("{vs_endpoint_name}")''')
  raise Exception(f"Timeout, your endpoint isn't ready yet: {vsc.get_endpoint(vs_endpoint_name)}")

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

VECTOR_SEARCH_ENDPOINT_NAME="dhbb_vs_endpoint"

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

Waiting for endpoint to be ready, this can take a few min... {'name': 'dhbb_vs_endpoint', 'creator': 'julio.chaves@fgv.br', 'creation_timestamp': 1741268454982, 'last_updated_timestamp': 1741268454982, 'endpoint_type': 'STANDARD', 'last_updated_user': 'julio.chaves@fgv.br', 'id': 'ecea8c48-118f-4552-9072-65263730a77b', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'dhbb_vs_endpoint', 'creator': 'julio.chaves@fgv.br', 'creation_timestamp': 1741268454982, 'last_updated_timestamp': 1741268454982, 'endpoint_type': 'STANDARD', 'last_updated_user': 'julio.chaves@fgv.br', 'id': 'ecea8c48-118f-4552-9072-65263730a77b', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'dhbb_vs_endpoint', 'creator': 'julio.chaves@fgv.br', 'creation_timestamp': 1741268454982, 'last_updated_timestamp': 1741268454982, 'endpoint_type': 'STANDARD',


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-prep-3.png?raw=true" style="float: right; margin-left: 10px" width="400px">


## Creating the Vector Search Index

Once the endpoint is created, all we now have to do is to as Databricks to create the index on top of the existing table. 

You just need to specify the text column and our embedding foundation model (`GTE`).  Databricks will build and synchronize the index automatically for us.

This can be done using the API, or in a few clicks within the Unity Catalog Explorer menu:

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/index_creation.gif?raw=true" width="600px">


In [0]:
def index_exists(vsc, endpoint_name, index_full_name):
    try:
        vsc.get_index(endpoint_name, index_full_name).describe()
        return True
    except Exception as e:
        if 'RESOURCE_DOES_NOT_EXIST' not in str(e):
            print(f'Unexpected error describing the index. This could be a permission issue.')
            raise e
    return False
    
def wait_for_index_to_be_ready(vsc, vs_endpoint_name, index_name):
  for i in range(180):
    idx = vsc.get_index(vs_endpoint_name, index_name).describe()
    index_status = idx.get('status', idx.get('index_status', {}))
    status = index_status.get('detailed_state', index_status.get('status', 'UNKNOWN')).upper()
    url = index_status.get('index_url', index_status.get('url', 'UNKNOWN'))
    if "ONLINE" in status:
      return
    if "UNKNOWN" in status:
      print(f"Can't get the status - will assume index is ready {idx} - url: {url}")
      return
    elif "PROVISIONING" in status:
      if i % 40 == 0: print(f"Waiting for index to be ready, this can take a few min... {index_status} - pipeline url:{url}")
      time.sleep(10)
    else:
        raise Exception(f'''Error with the index - this shouldn't happen. DLT pipeline might have been killed.\n Please delete it and re-run the previous cell: vsc.delete_index("{index_name}, {vs_endpoint_name}") \nIndex details: {idx}''')
  raise Exception(f"Timeout, your index isn't ready yet: {vsc.get_index(index_name, vs_endpoint_name)}")

def wait_for_model_serving_endpoint_to_be_ready(ep_name):
    from databricks.sdk import WorkspaceClient
    from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate
    import time

    # TODO make the endpoint name as a param
    # Wait for it to be ready
    w = WorkspaceClient()
    state = ""
    for i in range(200):
        state = w.serving_endpoints.get(ep_name).state
        if state.config_update == EndpointStateConfigUpdate.IN_PROGRESS:
            if i % 40 == 0:
                print(f"Waiting for endpoint to deploy {ep_name}. Current state: {state}")
            time.sleep(10)
        elif state.ready == EndpointStateReady.READY:
          print('endpoint ready.')
          return
        else:
          break
    raise Exception(f"Couldn't start the endpoint, timeout, please check your endpoint for more details: {state}")

In [0]:
%sql
ALTER TABLE dhbb.bronze.dhbb_bronze SET TBLPROPERTIES (delta.enableChangeDataFeed = true);


In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

#The table we'd like to index
source_table_fullname = f"dhbb.bronze.dhbb_bronze"
# Where we want to store our index
vs_index_fullname = f"dhbb.bronze.dhbb_bronze_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED",
    primary_key="id",
    embedding_source_column='content', #The column containing our text
    embedding_model_endpoint_name='databricks-gte-large-en' #The embedding endpoint used to create the embeddings
  )
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")

Creating index dhbb.bronze.dhbb_bronze_vs_index on endpoint dhbb_vs_endpoint...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://fgv-pocs-genie.cloud.databricks.com/explore/data/dhbb/bronze/dhbb_bronze_vs_index', 'indexed_row_count': 0, 'ready': False, 'index_url': 'fgv-pocs-genie.cloud.databricks.com/api/2.0/vector-search/endpoints/dhbb_vs_endpoint/indexes/dhbb.bronze.dhbb_bronze_vs_index'} - pipeline url:fgv-pocs-genie.cloud.databricks.com/api/2.0/vector-search/endpoints/dhbb_vs_endpoint/indexes/dhbb.bronze.dhbb_bronze_vs_index
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INITIAL_SNAPSHOT', 'message': 'Index is currently is in the process of syncing initial data. Check latest status: https://fgv-pocs-genie.cloud.databricks.com/explore/data/dhbb/bronze/dhbb_bronze_vs_index', 'indexed_row_count': 1940, 'provisioni

## Searching for relevant content

That's all we have to do. Databricks will automatically capture and synchronize new entries in your table with the index.

Note that depending on your dataset size and model size, index creation can take a few seconds to start and index your embeddings.

Let's give it a try and search for similar content.

*Note: `similarity_search` also support a filters parameter. This is useful to add a security layer to your RAG system: you can filter out some sensitive content based on who is doing the call (for example filter on a specific department based on the user preference).*


In [0]:
question = "Qual é o nome do Tiririca?"

results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
  query_text=question,
  columns=["content"],
  num_results=1)
docs = results.get('result', {}).get('data_array', [])
docs

[['---\ntitle: TIRIRICA\nnatureza: biográfico\nsexo: m\ncargos: \n - dep. fed. SP 2011-2015 \n---\n\n«Francisco Everardo Oliveira Silva» nasceu em Itapipoca (CE) no dia 1º de maio \nde 1965, filho de Fernando Oliveira Silva e Maria Alice da Silva.\n\nHumorista, ganhou o apelido Tiririca ainda criança. Aos 8 anos começou a \ntrabalhar no circo de sua cidade natal. Com o tempo foi ganhando destaque e \nlevou suas apresentações do interior do Nordeste para Fortaleza e Região \nMetropolitana. Ganhou maior repercussão a partir de 1997, quando lançado seu o \nseu primeiro CD, intitulado «Florentina», cuja música de mesmo nome fez grande \nsucesso no país na década de 90 e levou o disco a atingir índices recordes de \nvendagem. Após a repercussão de sua primeira gravação, participou de diversos \nprogramas televisivos, que possibilitaram, posteriormente, o lançamento de \noutros três CDs. \n\nTrabalhou também na Rede Manchete, onde apresentou um programa infantil, e, \napós isso foi contratad

# Deploy our chatbot model with RAG using DBRX

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-chain-1.png?raw=true" style="float: right" width="500px">

We've seen how Databricks makes it easy to ingest and prepare your documents, and deploy a Vector Search index on top of it with just clicks.

Now that our Vector Searc index is ready, let's deploy a langchain application.

## Configuring our Chain parameters

As any appliaction, a RAG chain needs some configuration for each environement (ex: different catalog for test/prod environement). 

Databricks makes this easy with Chain Configurations. You can use this object to configure any value within your app, including the different system prompts and make it easy to test and deploy newer version with better prompt.

In [0]:
# For this first basic demo, we'll keep the configuration as a minimum. In real app, you can make all your RAG as a param (such as your prompt template to easily test different prompts!)
chain_config = {
    "llm_model_serving_endpoint_name": "databricks-meta-llama-3-1-70b-instruct",  # the foundation model we want to use
    "vector_search_endpoint_name": VECTOR_SEARCH_ENDPOINT_NAME,  # the endoint we want to use for vector search
    "vector_search_index": f"dhbb.bronze.dhbb_bronze_vs_index",
    "llm_prompt_template": """Você é um assistente que responde perguntas sobre o acervo histório de pessoas políticas no Brasil. Use os seguintes verbetes como  contexto para responder à pergunta. Alguns pedaços de contexto podem ser irrelevantes, nesse caso você não deve usá-los para formar a resposta. Responda em português Brasil por padrão.\n\nContext: {context}""",
}

### Building our Langchain retriever

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-chain-2.png?raw=true" style="float: right" width="500px">

Let's start by building our Langchain retriever. 

It will be in charge of:

* Creating the input question (our Managed Vector Search Index will compute the embeddings for us)
* Calling the vector search index to find similar documents to augment the prompt with 

Databricks Langchain wrapper makes it easy to do in one step, handling all the underlying logic and API call for you.


In [0]:
from pyspark.sql.functions import pandas_udf
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf, length, pandas_udf
import os
import mlflow
import yaml
from typing import Iterator
from mlflow import MlflowClient
mlflow.set_registry_uri('databricks-uc')

# Workaround for a bug fix that is in progress
mlflow.spark.autolog(disable=True)

import warnings
warnings.filterwarnings("ignore")
# Disable MLflow warnings
import logging
logging.getLogger('mlflow').setLevel(logging.ERROR)

In [0]:
def display_txt_as_html(txt):
    txt = txt.replace('\n', '<br/>')
    displayHTML(f'<div style="max-height: 150px">{txt}</div>')

In [0]:
from databricks.vector_search.client import VectorSearchClient
from databricks_langchain.vectorstores import DatabricksVectorSearch
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

## Enable MLflow Tracing
mlflow.langchain.autolog()

## Load the chain's configuration
model_config = mlflow.models.ModelConfig(development_config=chain_config)

## Turn the Vector Search index into a LangChain retriever
vector_search_as_retriever = DatabricksVectorSearch(
    endpoint=model_config.get("vector_search_endpoint_name"),
    index_name=model_config.get("vector_search_index"),
    columns=["file_name", "content"],
).as_retriever(search_kwargs={"k": 3})

# Method to format the docs returned by the retriever into the prompt (keep only the text from chunks)
def format_context(docs):
    chunk_contents = [f"Passage: {d.page_content}\n" for d in docs]
    return "".join(chunk_contents)

#Let's try our retriever chain:
relevant_docs = (vector_search_as_retriever | RunnableLambda(format_context)| StrOutputParser()).invoke('Quem foi José Siqueira?')

display_txt_as_html(relevant_docs)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Passage: --- title: SIQUEIRA, Darci natureza: biográfico sexo: m cargos: - militar - dir. DASP 1974-1979 --- «Darci Duarte de Siqueira» nasceu no Rio de Janeiro, então Distrito Federal, no dia 7 de janeiro de 1925, filho de Joaquim Rodrigues Alves de Siqueira e Jovina Duarte de Siqueira. Sentou praça em março de 1942, ingressando no ano seguinte na Escola Militar de Resende, de onde saiu aspirante em 1946. Promovido a segundo-tenente em junho de 1947 e a primeiro-tenente em junho de 1949, chegou a capitão em abril de 1952. Psicólogo, foi chefe da seção de psicotécnica da Academia Militar das Agulhas Negras (AMAN) de 1958 a 1964. Em 1965 foi transferido para a reserva e assumiu a chefia da divisão de recrutamento e seleção da Petrobras. Em 1967 tornou-se chefe do serviço de pessoal da mesma empresa, cargo que desempenhou até 1974, quando foi nomeado pelo presidente Ernesto Geisel (1974-1979) diretor-geral do Departamento Administrativo do Serviço Público (DASP), sucedendo a Glauco Lessa de Abreu e Silva. Durante sua gestão no DASP concedeu uma entrevista à revista «Veja», em agosto de 1976, na qual atribuiu a origem da mordomia nos cargos públicos, em Brasília, à falta de infra-estrutura daquela cidade à época da transferência da capital, implicando a existência de um salário indireto aos servidores do Estado para lá transferidos. Segundo ele, perdeu-se o controle sobre esta folha de pagamentos, que não se justificava mais em face do desenvolvimento da capital. Em abril de 1979, com o fim do governo Geisel - substituído na presidência da República pelo general João Batista Figueiredo (1979-1985) -, Siqueira retornou ao seu antigo cargo na Petrobras, tendo sido sucedido no DASP por José Carlos Freire. Este segundo período de trabalho na estatal durou 11 anos. Em março de 1990, com a posse de Fernando Collor de Melo (1990-1992) na presidência da República, deixou a empresa, abandonando desde então a vida pública. Faleceu na cidade do Rio de Janeiro, em 5 de abril de 1999. Era casado com Regina Coelho de Sousa de Siqueira, com quem teve três filhos. Passage: --- title: SIQUEIRA, Jair natureza: biográfico sexo: m cargos: - dep. fed. MG 1995-1996 --- «Jair Siqueira» nasceu em Paulistas (MG) no dia 30 de junho de 1936, filho de José Cândido Siqueira e de Emília Soares Ferreira. Depois de fazer os primeiros estudos em sua cidade natal, transferiu-se para São Paulo, onde, em 1960, tornou-se funcionário da Gessy Lever e, a partir de 1967, gerente do Laboratório Andrômaco. Em 1971, iniciou o curso de direito na Universidade de São Paulo (USP), que concluiria em 1975. Em 1973, tornou-se diretor da Indústria de Fitas Jomak. Em 1976 fundou e assumiu a presidência da empresa Sigra, também em São Paulo. Em 1984, fundou o jornal «Sul Gerais» em Pouso Alegre (MG), para onde se transferiu, quando abriu uma filial da companhia. Em 1985, foi fundador e presidente do Centro das Indústrias do Médio Sapucaí e, no ano seguinte, tornou-se professor de teoria geral do estado na Faculdade de Direito do Sul de Minas, em Pouso Alegre. Em 1987, participou do seminário Brasil-França sobre privatização de empresas públicas, realizado em Paris, e, no ano seguinte, filiou-se ao Partido da Frente Liberal (PFL) na cidade mineira. Eleito prefeito de Pouso Alegre em outubro de 1988, não deixou suas atividades empresariais. Em 1989 tornou-se secretário da Fundação Sul Mineira de Ensino, administradora da Faculdade de Direito do Sul de Minas e, em 1991, foi fundador e diretor da empresa J. S. Têxtil Aviamentos e Tecidos. Deixou a prefeitura no final de seu mandato, no início de 1993. Em outubro de 1994, obteve uma cadeira na Câmara dos Deputados por Minas Gerais, com a maioria dos votos provenientes de sua base eleitoral no sul do estado. Participou dos trabalhos legislativos como membro titular da Comissão de Constituição e Justiça. Em 1995, trocou o PFL pelo Partido Progressista Brasileiro (PPB). Nas votações das emendas constitucionais propostas pelo governo Ferna

Trace(request_id=tr-53f6379c3396495eb1802aa53657f327)

You can see in the results that Databricks automatically trace your chain details and you can debug each steps and review the documents retrieved.

## Building Databricks Chat Model to query our demo's Foundational LLM

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-chain-3.png?raw=true" style="float: right" width="500px">

Our chatbot will be using Meta's Llama open source model. However, it could be utilized with DBRX (_pictured_), or any other LLMs served on Databricks.  

Other types of models that could be utilized include:

- Databricks Foundation models (_what we will use by default in this demo_)
- Your organization's custom, fine-tuned model
- An external model provider (_such as Azure OpenAI_)


In [0]:
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain.chat_models import ChatDatabricks
from operator import itemgetter

prompt = ChatPromptTemplate.from_messages(
    [  
        ("system", model_config.get("llm_prompt_template")), # Contains the instructions from the configuration
        ("user", "{question}") #user's questions
    ]
)

# Our foundation model answering the final prompt
model = ChatDatabricks(
    endpoint=model_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 1500}
)

#Let's try our prompt:
answer = (prompt | model | StrOutputParser()).invoke({'question':'Quem foi José Siqueira?', 'context': ''})
display_txt_as_html(answer)

Até a data do meu último conhecimento em abril de 2023, não há informações suficientes para identificar um personagem histórico ou político proeminente chamado José Siqueira no contexto da história política do Brasil. É possível que José Siqueira seja uma figura menos conhecida, um político local, ou talvez as informações sobre ele não sejam amplamente disponíveis ou documentadas. Se você tiver mais detalhes ou contexto sobre quem é José Siqueira ou em que período ele foi ativo, poderia me ajudar a fornecer uma resposta mais precisa.

Trace(request_id=tr-afa5bd919b5241458d9ad1c2e2116527)


## Putting it together in a final chain, supporting the standard Chat Completion format

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-basic-chain-4.png?raw=true" style="float: right" width="500px">


Let's now merge the retriever and the model in a single Langchain chain.

We will use a custom langchain template for our assistant to give proper answer.

We will make sure our chain support the standard Chat Completion API input schema : `{"messages": [{"role": "user", "content": "What is Retrieval-augmented Generation?"}]}`

Make sure you take some time to try different templates and adjust your assistant tone and personality for your requirement.

*Note that we won't support history in this first version, and will only take the last message as the question. See the advanced demo for a more complete example.*

In [0]:
# Return the string contents of the most recent messages: [{...}] from the user to be used as input question
def extract_user_query_string(chat_messages_array):
    return chat_messages_array[-1]["content"]

# RAG Chain
chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_user_query_string),
        "context": itemgetter("messages")
        | RunnableLambda(extract_user_query_string)
        | vector_search_as_retriever
        | RunnableLambda(format_context),
    }
    | prompt
    | model
    | StrOutputParser()
)


#### Databricks will track all the chain for you

<img src="https://ai-cookbook.io/_images/mlflow_trace2.gif" width="600px" style="float: right; margin-left: 10px">

As you can see in the cell result below, Databricks automatically trace the chain call. 

This makes it super easy to debug and improve your chain!

In [0]:
# Let's give it a try:
input_example = {"messages": [ {"role": "user", "content": "Qual é o nome do Tiririca?"}]}
answer = chain.invoke(input_example)
print(answer)

O nome do Tiririca é Francisco Everardo Oliveira Silva.


Trace(request_id=tr-9447b379cfd74038b144e34ee45e27ac)

## Deploy a RAG Chain to a web-based UI for stakeholder feedback

Our chain is now ready! 

Let's first register the Rag Chain model to MLFlow and Unity Catalog, and then use Agent Framework to deploy to the Agent Evaluation stakeholder review application which is backed by a scalable, production-ready Model Serving endpoint.

In [0]:
from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint
# Log the model to MLflow
with mlflow.start_run(run_name="dhbb_rag_bot"):
  logged_chain_info = mlflow.langchain.log_model(
          #Note: In classical ML, MLflow works by serializing the model object.  In generative AI, chains often include Python packages that do not serialize.  Here, we use MLflow's new code-based logging, where we saved our chain under the chain notebook and will use this code instead of trying to serialize the object.
          lc_model=os.path.join(os.getcwd(), 'chain'),  # Chain code file e.g., /path/to/the/chain.py 
          model_config=chain_config, # Chain configuration 
          artifact_path="chain", # Required by MLflow, the chain's code/config are saved in this directory
          input_example=input_example,
          example_no_conversion=True,  # Required by MLflow to use the input_example as the chain's schema,
          # Specify resources for automatic authentication passthrough
          resources=[
            DatabricksVectorSearchIndex(index_name=model_config.get("vector_search_index")),
            DatabricksServingEndpoint(endpoint_name=model_config.get("llm_model_serving_endpoint_name"))
          ]
      )

MODEL_NAME = "dhbb_rag_demo"
MODEL_NAME_FQN = f"dhbb.bronze.{MODEL_NAME}"
# Register to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=MODEL_NAME_FQN)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

Successfully registered model 'dhbb.bronze.dhbb_rag_demo'.


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

Created version '1' of model 'dhbb.bronze.dhbb_rag_demo'.


Let's now deploy the Mosaic AI **Agent Evaluation review application** using the model we just created!

In [0]:
from databricks import agents
# Deploy to enable the Review APP and create an API endpoint
# Note: scaling down to zero will provide unexpected behavior for the chat app. Set it to false for a prod-ready application.
deployment_info = agents.deploy(MODEL_NAME_FQN, model_version=uc_registered_model_info.version, scale_to_zero=True)

instructions_to_reviewer = f"""## Instruções para testar o chat bot para fazer perguntas aos verbetes disponíveis no DHBB

Suas contribuições são inestimáveis ​​para a equipe de desenvolvimento do hackathon ChatFGV . Ao fornecer feedback e correções detalhadas, você nos ajuda a corrigir problemas e melhorar a qualidade geral do aplicativo. Contamos com sua experiência para identificar quaisquer lacunas ou áreas que precisem de aprimoramento."""

# Add the user-facing instructions to the Review App
agents.set_review_instructions(MODEL_NAME_FQN, instructions_to_reviewer)

wait_for_model_serving_endpoint_to_be_ready(deployment_info.endpoint_name)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-5058f163-34b5-4763-8052-016c8700fd68/lib/python3.10/site-packages/mlflow/pyfunc/utils/data_validation.py:166: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Successfully registered model 'dhbb.bronze.feedback'.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '1' of model 'dhbb.bronze.feedback'.



    Deployment of dhbb.bronze.dhbb_rag_demo version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://fgv-pocs-genie.cloud.databricks.com/ml/endpoints/agents_dhbb-bronze-dhbb_rag_demo
    Review App: https://fgv-pocs-genie.cloud.databricks.com/ml/review/dhbb.bronze.dhbb_rag_demo/1?o=4008450564360146
    Monitoring guide: https://docs.databricks.com/en/generative-ai/agent-evaluation/evaluating-production-traffic.html?utm_source=agents.deploy&utm_medium=python-sdk
    
Waiting for endpoint to deploy agents_dhbb-bronze-dhbb_rag_demo. Current state: EndpointState(config_update=<EndpointStateConfigUpdate.IN_PROGRESS: 'IN_PROGRESS'>, ready=<EndpointStateReady.NOT_READY: 'NOT_READY'>)
Waiting for endpoint to deploy agents_dhbb-bronze-dhbb_rag_demo. Current state: EndpointState(config_update=<EndpointStateConfigUpdate.IN_PROGRESS: 'IN_PROGRESS'>, ready=<EndpointStateReady.NOT_READY: 'NOT_REA

# Use the Mosaic AI Agent Evaluation to evaluate your RAG applications

## Chat with your bot and build your validation dataset!

Our Chat Bot is now live. Databricks provides a built-in chatbot application that you can use to test the chatbot and give feedbacks on its answer.

You can easily give access to external domain experts and have them test and review the bot.  **Your domain experts do NOT need to have Databricks Workspace access** - you can assign permissions to any user in your SSO if you have enabled [SCIM](https://docs.databricks.com/en/admin/users-groups/scim/index.html)

This is a critical step to build or improve your evaluation dataset: have users ask questions to your bot, and provide the bot with output answer when they don't answer properly.

Your Chatbot is automatically capturing all stakeholder questions and bot responses, including an MLflow trace for each, into Delta Tables in your Lakehouse. On top of that, Databricks makes it easy to track feedback from your end user: if the chatbot doesn't give a good answer and the user gives a thumbdown, their feedback is included in the Delta Tables.

Once your eval dataset is ready, you'll then be able to leverage it for offline evaluation to measure your new chatbot performance, and also potentially to Fine Tune your model.
<br/>

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/eval-framework.gif?raw=true" width="1000px">


In [0]:
print(f"\n\nReview App URL to share with your stakeholders: {deployment_info.review_app_url}")



Review App URL to share with your stakeholders: https://fgv-pocs-genie.cloud.databricks.com/ml/review/dhbb.bronze.dhbb_rag_demo/1?o=4008450564360146
